In [ ]:
import requests
import json
import pyttsx3
import speech_recognition as sr
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_current_datetime():
    """Get the current date and time."""
    now = datetime.now()
    return now.strftime("%d/%m/%Y %H:%M")

# Get the current date and time
current_time = get_current_datetime()

# Mistral LLM interaction
def query_mistral(prompt):
    try:
        data = {
            "model": "mistral",
            "messages": [{"role": "user", "content": prompt}],
            "stream": False
        }
        response = requests.post("http://localhost:11434/api/chat", json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error querying Mistral: {e}")
        return None

def speak_text(text):
    """Use pyttsx3 to convert text to speech."""
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('volume', 0.9)
    engine.say(text)
    engine.runAndWait()

def get_voice_input():
    """Capture voice input with explicit stop command."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening... Say 'stop recording' to end.")
        full_text = ""

        while True:
            try:
                audio = recognizer.listen(source, timeout=None)
                text = recognizer.recognize_google(audio).lower()
                print(f"Heard: {text}")

                if "stop recording" in text:
                    print("Stopping recording.")
                    break

                full_text += text + ". "

            except sr.UnknownValueError:
                print("Could not understand the audio. Please try again.")
            except sr.RequestError as e:
                print(f"Could not request results from STT service; {e}")
                return None

        return full_text.strip()

def authenticate_google():
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file('credentials_calendar_automation.json', SCOPES)
    creds = flow.run_local_server(port=0)
    return build('calendar', 'v3', credentials=creds)

def create_event(service, title, date, time):
    if not date or date.strip() == "":
        date = get_current_datetime().split(" ")[0]

    event_start = datetime.strptime(f"{date} {time}", "%d/%m/%Y %H:%M")
    event_end = event_start + timedelta(hours=1)

    event = {
        'summary': title,
        'start': {
            'dateTime': event_start.isoformat(),
            'timeZone': 'America/Sao_Paulo',
        },
        'end': {
            'dateTime': event_end.isoformat(),
            'timeZone': 'America/Sao_Paulo',
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 30},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }

    event_result = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event_result.get('htmlLink')}")
    return event_result.get('htmlLink')

def main():
    service = authenticate_google()

    while True:
        print("Speak your request (or type 'exit' to quit):")
        user_input = get_voice_input()
        if user_input is None:
            continue

        if "exit" in user_input.lower():
            print("Exiting program.")
            break

        prompt = f"Considering today's date and time ({current_time}) Extract action ,time(hh:mm), date(dd/mm/yyyy), and description from: {user_input}, output only the following format Appointment title: title / Date: dd/mm/yyyy / Time: hh:mm"
        parsed_json = query_mistral(prompt)

        if parsed_json:
            try:
                ai_reply = parsed_json.get("message", {}).get("content", "")
                print("\nAI Response:")
                print(ai_reply)

                if ai_reply:
                    lines = ai_reply.split("/ ")
                    appointment_title = lines[0].replace("Appointment title: ", "").strip()
                    appointment_date = lines[1].replace("Date: ", "").strip()
                    appointment_time = lines[2].replace("Time: ", "").strip()

                    if "(" in appointment_time:
                        appointment_time = appointment_time.split("(")[0].strip()

                    if not appointment_date:
                        appointment_date = get_current_datetime().split(" ")[0]

                    print("\nParsed Variables:")
                    print(f"Title: {appointment_title}")
                    print(f"Date: {appointment_date if appointment_date else 'Today'}")
                    print(f"Time: {appointment_time}")

                    event_link = create_event(service, appointment_title, appointment_date, appointment_time)

                    success_message = f"Your appointment titled '{appointment_title}' has been successfully created."
                    print(success_message)
                    speak_text(success_message)

                else:
                    print("No valid response from AI.")

            except KeyError as e:
                print(f"Error parsing response: {e}")
                print("Unexpected response format. Full response:")
                print(parsed_json)
        else:
            print("Failed to parse input. Please try again.")

if __name__ == '__main__':
    main()
